# Lesson 6: Shipping as a web API

In [1]:
import "dotenv/config";

[Module: null prototype] { default: {} }

In [2]:
import { 
  loadAndSplitChunks, 
  initializeVectorstoreWithDocuments 
} from "./lib/helpers.ts";

const splitDocs = await loadAndSplitChunks({
  chunkSize: 1536,
  chunkOverlap: 128,
});

const vectorstore = await initializeVectorstoreWithDocuments({
  documents: splitDocs,
});

const retriever = vectorstore.asRetriever();

In [3]:
import { 
  createDocumentRetrievalChain, 
  createRephraseQuestionChain 
} from "./lib/helpers.ts";

const documentRetrievalChain = createDocumentRetrievalChain();
const rephraseQuestionChain = createRephraseQuestionChain();

In [4]:
import { ChatPromptTemplate, MessagesPlaceholder } from "@langchain/core/prompts";

const ANSWER_CHAIN_SYSTEM_TEMPLATE = `You are an experienced researcher,
expert at interpreting and answering questions based on provided sources.
Using the below provided context and chat history, 
answer the user's question to the best of your ability
using only the resources provided. Be verbose!

<context>
{context}
</context>`;

const answerGenerationChainPrompt = ChatPromptTemplate.fromMessages([
  ["system", ANSWER_CHAIN_SYSTEM_TEMPLATE],
  new MessagesPlaceholder("history"),
  [
    "human", 
    `Now, answer this question using the previous context and chat history:
  
    {standalone_question}`
  ]
]);

In [5]:
import { 
  RunnablePassthrough, 
  RunnableSequence 
} from "@langchain/core/runnables";
import { ChatOpenAI } from "@langchain/openai";

const conversationalRetrievalChain = RunnableSequence.from([
  RunnablePassthrough.assign({
    standalone_question: rephraseQuestionChain,
  }),
  RunnablePassthrough.assign({
    context: documentRetrievalChain,
  }),
  answerGenerationChainPrompt,
  new ChatOpenAI({ modelName: "gpt-3.5-turbo-1106" }),
]);

In [6]:
import { HttpResponseOutputParser } from "langchain/output_parsers";

// "text/event-stream" is also supported
const httpResponseOutputParser = new HttpResponseOutputParser({
  contentType: "text/plain"
});

In [7]:
import { RunnableWithMessageHistory } from "@langchain/core/runnables"; 
import { ChatMessageHistory } from "langchain/stores/message/in_memory";

const messageHistory = new ChatMessageHistory();

const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: (_sessionId) => messageHistory,
  historyMessagesKey: "history",
  inputMessagesKey: "question",
}).pipe(httpResponseOutputParser);

Additionally, we'll want to bear in mind that users should not share chat histories, and we should create a new history object per session:

In [8]:
const messageHistories = {};

const getMessageHistoryForSession = (sessionId) => {
    if (messageHistories[sessionId] !== undefined) {
        return messageHistories[sessionId];
    } 
    const newChatSessionHistory = new ChatMessageHistory();
    messageHistories[sessionId] = newChatSessionHistory;
    return newChatSessionHistory;
};

We'll recreate our final chain with this new method:

In [9]:
const finalRetrievalChain = new RunnableWithMessageHistory({
  runnable: conversationalRetrievalChain,
  getMessageHistory: getMessageHistoryForSession,
  inputMessagesKey: "question",
  historyMessagesKey: "history",
}).pipe(httpResponseOutputParser);

In [10]:
const port = 8087;

In [11]:
const handler = async (request: Request): Response => {
  const body = await request.json();
  const stream = await finalRetrievalChain.stream({
    question: body.question
  }, { configurable: { sessionId: body.session_id } });

  return new Response(stream, { 
    status: 200,
    headers: {
      "Content-Type": "text/plain"
    },
  });
};

In [12]:
Deno.serve({ port }, handler);

Listening on http://localhost:8087/


{
  finished: Promise { <pending> },
  shutdown: [AsyncFunction: shutdown],
  ref: [Function: ref],
  unref: [Function: unref],
  [Symbol(Symbol.asyncDispose)]: [Function: [Symbol.asyncDispose]]
}

In [13]:
const decoder = new TextDecoder();

// readChunks() reads from the provided reader and yields the results into an async iterable
function readChunks(reader) {
  return {
    async* [Symbol.asyncIterator]() {
      let readResult = await reader.read();
      while (!readResult.done) {
        yield decoder.decode(readResult.value);
        readResult = await reader.read();
      }
    },
  };
}

const sleep = async () => {
  return new Promise((resolve) => setTimeout(resolve, 500));
}

In [14]:
const response = await fetch(`http://localhost:${port}`, {
    method: "POST",
    headers: {
        "content-type": "application/json",
    },
    body: JSON.stringify({
        question: "What are the prerequisites for this course?",
        session_id: "1", // Should randomly generate/assign
    })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: The prer
CHUNK: equisites
CHUNK:  for this 
CHUNK: course, 
CHUNK: as outlined in t
CHUNK: he provided conte
CHUNK: xt, include f
CHUNK: amiliarity w
CHUNK: ith basic 
CHUNK: probability a
CHUNK: nd statist
CHUNK: ics, as wel
CHUNK: l as basic linear
CHUNK:  algebra. Fo
CHUNK: r statistics, it is
CHUNK:  mentioned t
CHUNK: hat most 
CHUNK: undergra
CHUNK: duate statistics class
CHUNK: , like St
CHUNK: at 116 taught
CHUNK:  at Stanford,
CHUNK:  will be more t
CHUNK: han enough. The 
CHUNK: expectation
CHUNK:  is that students ar
CHUNK: e familiar with random
CHUNK:  variable
CHUNK: s, expectation, variance, and oth
CHUNK: er relat
CHUNK: ed concepts.
CHUNK:  To refresh
CHUNK:  the material,
CHUNK:  some discussion 
CHUNK: sections
CHUNK:  will go over th
CHUNK: e prerequisites as a
CHUNK:  refresher course.

Regard
CHUNK: ing linear algebra,
CHUNK:  the expectation is a basic 
CHUNK: familiarity assumed with 
CHUNK: concepts such
CHUNK:  as matrices, v
CHUNK: ectors, matri
CHUNK:

In [15]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "Can you list them in bullet point format?",
    session_id: "1", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Certain
CHUNK: ly! Base
CHUNK: d on the provi
CHUNK: ded context a
CHUNK: nd chat hi
CHUNK: story, the prere
CHUNK: quisites for
CHUNK:  the course can be 
CHUNK: listed in 
CHUNK: bullet p
CHUNK: oint format
CHUNK:  as follows:
CHUNK: 

- Famil
CHUNK: iarity w
CHUNK: ith basic proba
CHUNK: bility and stati
CHUNK: stics, including
CHUNK:  concepts such as 
CHUNK: random vari
CHUNK: ables, e
CHUNK: xpectation,
CHUNK:  and variance. An
CHUNK:  example of a s
CHUNK: uitable p
CHUNK: rerequisite course is St
CHUNK: at 116 ta
CHUNK: ught at
CHUNK:  Stanford.
- Bas
CHUNK: ic knowl
CHUNK: edge of line
CHUNK: ar algeb
CHUNK: ra, encomp
CHUNK: assing matrices, 
CHUNK: vectors,
CHUNK:  matrix multiplic
CHUNK: ation, matrix
CHUNK:  inverse, and pot
CHUNK: entially 
CHUNK: eigenvectors. P
CHUNK: rerequisite courses at Stanford t
CHUNK: hat cover these topics include Math 51, 103, Math 113, or CS205.
- Basic knowledge of computer science and computer skills, including an understanding of big

In [16]:
const response = await fetch(`http://localhost:${port}`, {
  method: "POST",
  headers: {
    "content-type": "application/json",
  },
  body: JSON.stringify({
    question: "What did I just ask you?",
    session_id: "2", // Should randomly generate/assign
  })
});

// response.body is a ReadableStream
const reader = response.body?.getReader();

for await (const chunk of readChunks(reader)) {
  console.log("CHUNK:", chunk);
}

await sleep();

CHUNK: Based on the 
CHUNK: provided 
CHUNK: context and
CHUNK:  chat hist
CHUNK: ory, you di
CHUNK: d not ask a spe
CHUNK: cific que
CHUNK: stion that I 
CHUNK: can refer
CHUNK:  to in or
CHUNK: der to provide a respons
CHUNK: e. The context i
CHUNK: ncludes i
CHUNK: nformation 
CHUNK: about diff
CHUNK: erent co
CHUNK: urse-rel
CHUNK: ated topics b
CHUNK: eing discu
CHUNK: ssed, in
CHUNK: structions
CHUNK:  about forming study g
CHUNK: roups and pro
CHUNK: ject partners, reminders about the honor code, and a diverse audience in the class. However, I don't see a specific question from you in the provided context.

If you can provide me with the question you asked, I'd be happy to answer it to the best of my ability.
